## Outreachy-Bokeh Microtask for May 2023 cohort.
### The source for the data file can be found [here](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page) 

In [1]:
#Import all necessary libraries
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import FactorRange
from math import pi
from bokeh.palettes import Category10, Category20
from bokeh.transform import cumsum


In [2]:
#Load output_notebook to generate plots inline
output_notebook()

Loading BokehJS ...

### Data loading and transformation

In [3]:
#Load data file.
file = r"yellow_tripdata_2022-11.parquet"
df = pd.read_parquet(file)

#Load a random sample of the data file up to 10 rows 
df.sample(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1359682,1,2022-11-12 13:51:44,2022-11-12 14:24:06,1.0,12.90,99.0,N,250,258,1,43.2,0.0,0.5,0.00,6.55,0.3,50.55,0.0,0.0
2317832,2,2022-11-22 16:27:05,2022-11-22 16:33:43,1.0,1.15,1.0,N,75,74,1,6.5,1.0,0.5,1.24,0.00,0.3,9.54,0.0,0.0
3047403,2,2022-11-30 11:25:26,2022-11-30 11:51:01,2.0,1.51,1.0,N,236,161,1,15.5,0.0,0.5,4.70,0.00,0.3,23.50,2.5,0.0
2807471,2,2022-11-28 07:54:24,2022-11-28 07:57:59,1.0,1.08,1.0,N,238,142,1,5.0,0.0,0.5,2.49,0.00,0.3,10.79,2.5,0.0
2778869,2,2022-11-27 18:10:24,2022-11-27 18:14:40,1.0,0.59,1.0,N,79,4,2,4.5,0.0,0.5,0.00,0.00,0.3,7.80,2.5,0.0
3174117,1,2022-11-11 09:47:44,2022-11-11 10:03:01,NaN,0.00,NaN,None,142,152,0,14.7,0.0,0.5,2.70,0.00,0.3,22.70,NaN,NaN
1686529,2,2022-11-15 19:05:30,2022-11-15 19:21:57,1.0,2.10,1.0,N,229,142,1,12.0,1.0,0.5,3.26,0.00,0.3,19.56,2.5,0.0
1473060,2,2022-11-13 11:19:12,2022-11-13 11:36:00,2.0,2.78,1.0,N,239,161,2,13.0,0.0,0.5,0.00,0.00,0.3,16.30,2.5,0.0
986281,2,2022-11-09 13:21:16,2022-11-09 13:29:07,5.0,0.96,1.0,N,48,68,1,7.0,0.0,0.5,3.09,0.00,0.3,13.39,2.5,0.0
3114711,2,2022-11-30 21:41:02,2022-11-30 22:07:40,1.0,6.01,1.0,N,234,61,1,22.5,0.5,0.5,5.26,0.00,0.3,31.56,2.5,0.0


In [4]:
#Check the columns data types
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [5]:
# Write a function that cleans up the data
def tweak_df(df):
    """
    Cleans up the dataframe by doing the following:
    1. Replaces all blank passenger_counts column with zero (0).
    2. Replaces all blank RatecodeID columns with five(5) for 'Unknown'.
    3. Converts the passenger_counts, RatecodeID, VendorID and payment_type 
        column to 'uint8' to save memory space.
    4. Adds a trip_duration column that is the difference 
        between pickup and dropoff times.
    5. Drops all rows where the trip duration is less than one second.
    
    Args:
        df(pandas dataframe): This is the dataframe from the file used here.
        
    Returns:
        df(pandas dataframe): The cleaned up dataframe.
        
    """
    df = df.assign(
        passenger_count = lambda df: df.passenger_count.fillna(0).astype('uint8'),
        payment_type = lambda df: df.payment_type.astype('uint8'),
        VendorID = lambda df: df.VendorID.astype('uint8'),
        RatecodeID = lambda df: df.RatecodeID.fillna(5).astype('uint8'),
        trip_duration = lambda df: (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)
    )
    duration_filtered = df[df['trip_duration'].dt.total_seconds() < 1.0]
    df = df.drop(duration_filtered.index)
    return df

In [6]:
#Check cleaned-up data types
new_df = tweak_df(df)

new_df.dtypes

VendorID                           uint8
tpep_pickup_datetime      datetime64[ns]
tpep_dropoff_datetime     datetime64[ns]
passenger_count                    uint8
trip_distance                    float64
RatecodeID                         uint8
store_and_fwd_flag                object
PULocationID                       int64
DOLocationID                       int64
payment_type                       uint8
fare_amount                      float64
extra                            float64
mta_tax                          float64
tip_amount                       float64
tolls_amount                     float64
improvement_surcharge            float64
total_amount                     float64
congestion_surcharge             float64
airport_fee                      float64
trip_duration            timedelta64[ns]
dtype: object

### Data insights and visualisation

In [7]:
#Group by trip days and calculate the total amount made by weekday.
trip_days = new_df.tpep_pickup_datetime.dt.day_name()
day_total = pd.DataFrame(new_df.groupby(trip_days).total_amount.sum())

day_total


,total_amount
tpep_pickup_datetime,
Friday,9428629.68
Monday,9236579.29
Saturday,9664645.22
Sunday,8931626.15
Thursday,9547948.45
Tuesday,12344706.81
Wednesday,12542277.80


In [8]:
#Sort the grouped data by weekday
weekday_cat = pd.CategoricalDtype(
    categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    ordered=True)
day_total.index = day_total.index.astype(weekday_cat)
day_total = day_total.sort_values('tpep_pickup_datetime')
day_total = day_total.reset_index()

day_total

,tpep_pickup_datetime,total_amount
0,Monday,9236579.29
1,Tuesday,12344706.81
2,Wednesday,12542277.80
3,Thursday,9547948.45
4,Friday,9428629.68
5,Saturday,9664645.22
6,Sunday,8931626.15


In [9]:
#Plot a bar chart showing the amount made by weekday 
days = day_total['tpep_pickup_datetime']
amount = day_total['total_amount']

p = figure(title = 'Total amount made by weekday', 
           sizing_mode = 'stretch_width',
           x_range = FactorRange(factors=days),
           toolbar_location = None)

p.vbar(x=days, top=amount, width=0.9)
p.y_range.start = (5 * 10**6)
         

show(p)

In [10]:
#Insights on trip duration
# Average trip length is 18 minutes.
#Longest trip duration is 3 days and ~21 hours
new_df.trip_duration.describe()

count                      3249767
mean     0 days 00:18:08.630485508
std      0 days 00:48:57.664849398
min                0 days 00:00:01
25%                0 days 00:07:51
50%                0 days 00:13:02
75%                0 days 00:21:05
max                3 days 20:55:10
Name: trip_duration, dtype: object

In [11]:
#Sort the pickup times by hour of the day
pick_up_hour = new_df.tpep_pickup_datetime.dt.hour

# Group the hour values into four general times of the day
conditions = [pick_up_hour < 5, (pick_up_hour>=5) & (pick_up_hour<12), (pick_up_hour>=12) & (pick_up_hour<=18), pick_up_hour>18]
values = ['Midnight', 'Morning', 'Afternoon', 'Evening']

result = np.select(conditions, values)

pick_up_hour = pd.Series(result)

pick_up_hour

0          Midnight
1          Midnight
2          Midnight
3          Midnight
4          Midnight
             ...   
3249762     Evening
3249763     Evening
3249764     Evening
3249765     Evening
3249766     Evening
Length: 3249767, dtype: object

In [12]:
#Generate a pandas series for the pickup hour by frequency
pick_up_data = pick_up_hour.value_counts()
pick_up_data = pick_up_data.reset_index(name='Frequency').rename(columns={'index':'Hour'})

#Add columns for plot angle and color
pick_up_data['angle'] = pick_up_data['Frequency']/pick_up_data['Frequency'].sum() * 2*pi
pick_up_data['color'] = Category10[len(pick_up_data)]

pick_up_data

,Hour,Frequency,angle,color
0,Afternoon,1412172,2.730331,#1f77b4
1,Evening,863630,1.669765,#ff7f0e
2,Morning,755126,1.459981,#2ca02c
3,Midnight,218839,0.423109,#d62728


In [13]:
#Plot a pie chart for the pickup data showing the frequency.
p = figure(height=350,
           title="Pickup Frequency by hour of day",
           toolbar_location=None,
           tools="hover",
           tooltips="@Hour: @Frequency", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.35,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Hour', source=pick_up_data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [14]:
#Insights on tip amount
#Average tip amount is ~$3
new_df.tip_amount.describe()

count    3.249767e+06
mean     2.847096e+00
std      3.312256e+00
min     -8.160000e+01
25%      1.000000e+00
50%      2.220000e+00
75%      3.500000e+00
max      3.330000e+02
Name: tip_amount, dtype: float64

In [15]:
#Generate total tip amount data by time of day
tip_data = new_df.groupby(pick_up_hour).tip_amount.sum()
tip_data = tip_data.reset_index(name='Amount').rename(columns={'index':'Hour'})

#Add columns for plot angle and color
tip_data['angle'] = tip_data['Amount']/tip_data['Amount'].sum() * 2*pi
tip_data['color'] = Category20[len(tip_data)]

tip_data

,Hour,Amount,angle,color
0,Afternoon,4068015.19,2.765325,#1f77b4
1,Evening,2480272.17,1.686021,#aec7e8
2,Midnight,601632.35,0.408973,#ff7f0e
3,Morning,2093151.94,1.422867,#ffbb78


In [16]:
#Plot a pie chart showing tip amounts by time of day
p = figure(height=350,
           title="Total tip amount by hour of day",
           toolbar_location=None,
           tools="hover",
           tooltips="@Hour: @Amount", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.35,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Hour', source=tip_data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)